In [1]:
import numpy as np
import open3d as o3d
import os
import matplotlib.pyplot as plt

In [2]:
%matplotlib notebook

In [3]:
# BASE_PATH = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/pair_transform_dynseg_all/emptycities_preprocess/test"
BASE_PATH = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/pair_transform_dynseg_all/emptycities_preprocess/test/"
DYNAMIC_FOLDER = "d"
MASK_FOLDER = "m"
# STATIC_FOLDER = "static_preprocessed"
# MASK_IMG_FOLDER = "m_img_40_512"

DYNAMIC_PATH = os.path.join(BASE_PATH, DYNAMIC_FOLDER, "emptycities_testd15.npy")
MASK_PATH = os.path.join(BASE_PATH, MASK_FOLDER, "emptycities_testm15.npy")
# STATIC_PATH = os.path.join(BASE_PATH, STATIC_FOLDER, "15.npy")
# MASK_IMG_PATH = os.path.join(BASE_PATH, MASK_IMG_FOLDER)

In [4]:
dynamic_arr = np.load(DYNAMIC_PATH)
# static_arr  = np.load(STATIC_PATH)
mask_arr    = np.load(MASK_PATH)[:,:,:,::2]

In [5]:
dynamic_arr.shape, mask_arr.shape#, static_arr.shape,

((1274, 2, 64, 1024), (1274, 2, 64, 512))

In [6]:
def draw_pcd(pcd, where='mat_3d'):
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.grid()
        plt.show()
    elif where is 'mat_2d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1])
        plt.grid()
        plt.show()
        
def from_polar_np(velo):
    angles = np.linspace(0, np.pi * 2, velo.shape[-1])
    dist, z = velo[:, 0], velo[:, 1]
    x = np.cos(angles) * dist
    y = np.sin(angles) * dist
    out = np.stack([x,y,z], axis=1)
    return out.astype('float32')

In [7]:
dynamic_3d_arr = from_polar_np(dynamic_arr)
# static_3d_arr = from_polar_np(static_arr)
dynamic_3d_arr.shape

(1274, 3, 64, 1024)

In [8]:
def get_pcd(idx, pcd_arr, mask_arr=None):
    this_arr = pcd_arr[idx].reshape(3,-1).T
    this_pcd = o3d.geometry.PointCloud()
    this_pcd.points = o3d.utility.Vector3dVector(this_arr)

    if mask_arr is not None:
        single_clr_arr = mask_arr[idx,1].reshape(1,-1)
        clr_arr = np.concatenate([single_clr_arr, np.zeros(single_clr_arr.shape), np.zeros(single_clr_arr.shape)],axis=0).T
        this_pcd.colors = o3d.utility.Vector3dVector(clr_arr)
    return this_pcd

In [9]:
draw_pcd(get_pcd(200, dynamic_3d_arr, mask_arr), where='opn_nb')

JVisualizer with 1 geometries